# QUANTIZATION AWARE TRAINING

In [2]:
# ! pip install tensorflow-model-optimization

In [1]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot

In [10]:
CONFIGURATION = {
    "CLASS_NAMES" : ['angry', 'happy', 'sad'],
    "BATCH_SIZE" : 32,
    "IMAGE_SIZE" : 256,
    "LEARNING_RATE" : 0.01,
    "N_EPOCHS" : 20,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE" : 0.0,
    "N_FILTERS" : 6,
    "KERNEL_SIZE" : 3,
    "N_STRIDES" : 1,
    "POOL_SIZE" : 2,
    "N_DENSE_1" : 1024,
    "N_DENSE_2" : 128,
    "NUM_CLASSES" : 3,
    "PATCH_SIZE" : 16,
}

trainDirectory = "/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/train"
testDirectory = "/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/test"

trainDataset = tf.keras.utils.image_dataset_from_directory(
    trainDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.2,
    subset='training',
)

valDataset = tf.keras.utils.image_dataset_from_directory(
    trainDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.2,
    subset='validation',
)

testDataset = tf.keras.utils.image_dataset_from_directory(
    testDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=None,
    subset=None,
)

trainDataset = trainDataset.prefetch(tf.data.AUTOTUNE)
testDataset = testDataset.prefetch(tf.data.AUTOTUNE)
valDataset = valDataset.prefetch(tf.data.AUTOTUNE)

Found 6799 files belonging to 3 classes.
Using 5440 files for training.
Found 6799 files belonging to 3 classes.
Using 1359 files for validation.
Found 2278 files belonging to 3 classes.


In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(CONFIGURATION['IMAGE_SIZE'], CONFIGURATION['IMAGE_SIZE'], 3)),

    tf.keras.layers.Conv2D(filters=CONFIGURATION["N_FILTERS"], kernel_size=CONFIGURATION["KERNEL_SIZE"], strides=CONFIGURATION["N_STRIDES"], padding='valid', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=CONFIGURATION["POOL_SIZE"], strides=CONFIGURATION["N_STRIDES"]),

    tf.keras.layers.Conv2D(filters=CONFIGURATION["N_FILTERS"], kernel_size=CONFIGURATION["KERNEL_SIZE"], strides=CONFIGURATION["N_STRIDES"], padding='valid', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=CONFIGURATION["POOL_SIZE"], strides=CONFIGURATION["N_STRIDES"]),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(CONFIGURATION["N_DENSE_1"], activation='relu'),
    tf.keras.layers.Dense(CONFIGURATION["N_DENSE_2"], activation='relu'),
    tf.keras.layers.Dense(CONFIGURATION["NUM_CLASSES"], activation='softmax'),
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 254, 254, 6)       168       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 253, 253, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 251, 251, 6)       330       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 250, 250, 6)       0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 375000)            0         
                                                                 
 dense_18 (Dense)            (None, 1024)             

## Making The Full Model Quant Aware

In [16]:
quantAwareModel = tfmot.quantization.keras.quantize_model(model)

## Making The Model Quant Aware Layer-by-Layer

In [20]:
# Loading Pre-Trained Efficient Net B4 Model trained on ImageNet Dataset
backbone = tf.keras.applications.EfficientNetB4(
    include_top = False, # Include Classifier or Not
    weights = 'imagenet',
    input_shape = (CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"], 3)
)


x = tf.keras.layers.GlobalAveragePooling2D()(backbone.output)
x = tf.keras.layers.Dense(CONFIGURATION['N_DENSE_1'], activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(CONFIGURATION['N_DENSE_2'], activation='relu')(x)
output = tf.keras.layers.Dense(CONFIGURATION['NUM_CLASSES'], activation='softmax')(x)
 
EfficientNetModel = tf.keras.models.Model(backbone.input, output)

### Annotating The Layers To Quantize

In [23]:
def applyQuantizationToConvLayers(layer):
    if 'conv' in layer.name:
        return tfmot.quantization.keras.quantize_annotate_layer(layer)
    return layer

quantAwareEfficientNetModel = tf.keras.models.clone_model(
    EfficientNetModel,
    clone_function=applyQuantizationToConvLayers
)
quantAwareEfficientNetModel.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 rescaling_3 (Rescaling)     (None, 256, 256, 3)          0         ['input_9[0][0]']             
                                                                                                  
 normalization_3 (Normaliza  (None, 256, 256, 3)          7         ['rescaling_3[2][0]']         
 tion)                                                                                            
                                                                                                  
 tf.math.multiply_3 (TFOpLa  (None, 256, 256, 3)          0         ['normalization_3[2][0]'

# Quantizing The Model

In [24]:
quantAwareEfficientNetModel = tfmot.quantization.keras.quantize_apply(quantAwareEfficientNetModel)
quantAwareEfficientNetModel.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 rescaling_3 (Rescaling)     (None, 256, 256, 3)          0         ['input_9[0][0]']             
                                                                                                  
 normalization_3 (Normaliza  (None, 256, 256, 3)          7         ['rescaling_3[1][0]']         
 tion)                                                                                            
                                                                                                  
 tf.math.multiply_3 (TFOpLa  (None, 256, 256, 3)          0         ['normalization_3[1][0]'

In [26]:
lossFunction = tf.keras.losses.CategoricalCrossentropy()
METRICS = [tf.keras.metrics.CategoricalAccuracy(name="accuracy"), 
           tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="top_k_accuracy")]

quantAwareEfficientNetModel.compile(
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=CONFIGURATION["LEARNING_RATE"]),
    loss = lossFunction,
    metrics=METRICS
)

In [29]:
history = quantAwareEfficientNetModel.fit(
    trainDataset.take(10),
    validation_data = valDataset.take(1),
    epochs = CONFIGURATION['N_EPOCHS'],
    verbose=1,
)

Epoch 1/20
10/10 [==============================] - 91s 9s/step - loss: 8.7212 - accuracy: 0.3594 - top_k_accuracy: 0.7094 - val_loss: 39.4370 - val_accuracy: 0.1875 - val_top_k_accuracy: 0.5000
Epoch 2/20
10/10 [==============================] - 96s 10s/step - loss: 4.9578 - accuracy: 0.3250 - top_k_accuracy: 0.6781 - val_loss: 30.1846 - val_accuracy: 0.3750 - val_top_k_accuracy: 0.6250
Epoch 3/20
10/10 [==============================] - 100s 10s/step - loss: 3.3222 - accuracy: 0.4000 - top_k_accuracy: 0.6938 - val_loss: 20.0077 - val_accuracy: 0.3125 - val_top_k_accuracy: 0.5625
Epoch 4/20
10/10 [==============================] - 100s 10s/step - loss: 2.4304 - accuracy: 0.3406 - top_k_accuracy: 0.6687 - val_loss: 34.3175 - val_accuracy: 0.3750 - val_top_k_accuracy: 0.6250
Epoch 5/20
10/10 [==============================] - 101s 10s/step - loss: 2.1971 - accuracy: 0.3969 - top_k_accuracy: 0.6844 - val_loss: 30.2055 - val_accuracy: 0.2188 - val_top_k_accuracy: 0.5312
Epoch 6/20
10/10 [